# Rolex Listing Price Prediction based on model and complications

In [148]:
import pandas as pd
import numpy as np
import glob
import janitor
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

## Data Cleaning

In [122]:
files = glob.glob('data/result_df/*.csv')
dirty_df = pd.concat((pd.read_csv(file, index_col=0)
                for file in files)
              )

dirty_df = dirty_df.clean_names()
dirty_df.drop_duplicates(subset=['listing_code', 'reference_number'], inplace=True)
dirty_df.dropna(subset=['brand', 'model', 'listing_code', 'price', 'title', 'subtitle', 'case_diameter'], inplace=True)
dirty_df.reset_index(drop=True, inplace=True)


dirty_df.head()

,listing_code,brand,model,reference_number,movement,case_material,bracelet_material,year_of_production,condition,scope_of_delivery,...,thickness,lug_width,buckle_width,frequency,bracelet_thickness,submariner_kermit_ref_,day_date_ref_,datejust_reference_number,submariner_date_reference,reference
0,IJD7R3,Rolex,Datejust 41,126331 NEW UNWORN 2023 Wimbledon 41mm Jubilee,Automatic,Gold/Steel,Gold/Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOAOQ5,Rolex,Datejust 31,278271,Automatic,Gold/Steel,Gold/Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IJ9RY8,Rolex,Datejust 36,126231,Automatic,Gold/Steel,Gold/Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FDHJM3,Rolex,GMT-Master II,126710BLNR,Automatic,Steel,Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FFF9D3,Rolex,Explorer,124270,Automatic,Steel,Steel,2021,Very good\n(Worn with little to no signs of wear),"Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
# clean case_diameter
def is_convertible_to_int(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

convertible_mask = dirty_df['case_diameter'].str[:2].apply(is_convertible_to_int)

dirty_df = dirty_df[convertible_mask]

dirty_df['case_diameter'] = dirty_df['case_diameter'].str[:2].astype('int')


In [117]:
# add column of whether the price is negotiable
dirty_df.insert(loc=13, column='is_negotiable', value=dirty_df['price'].str.contains('Negotiable', case=False).astype(int))

In [124]:
# keep only CA$ in the `price` column
dirty_df['price'] = dirty_df['price'].str.extract('C\$([0-9,]+)')[0].str.replace(',', '')
dirty_df['price'] = pd.to_numeric(dirty_df['price'], errors='coerce')
dirty_df['price'].fillna(0, inplace=True)
dirty_df['price'] = dirty_df['price'].astype(int)

dirty_df = dirty_df.query('price != 0')

<>:2: DeprecationWarning: invalid escape sequence '\$'
<>:2: DeprecationWarning: invalid escape sequence '\$'
C:\Users\fohy2\AppData\Local\Temp\ipykernel_14912\1010010397.py:2: DeprecationWarning: invalid escape sequence '\$'
  dirty_df['price'] = dirty_df['price'].str.extract('C\$([0-9,]+)')[0].str.replace(',', '')


In [131]:
# add column of whether the year of production is approximated
dirty_df.insert(loc=8, column='year_is_approximated', value=dirty_df['year_of_production'].str.contains('Approximation', case=False).astype(int))

# Clean year of production
dirty_df['year_of_production'] = dirty_df['year_of_production'].apply(lambda x: x[:4] if x != 'Unknown' else x)

In [135]:
# simplify the location to country only
dirty_df['country'] = dirty_df['location'].str.split(',').str[0]

Save the cleaned data locally

In [136]:
rolex_df = dirty_df
rolex_df.to_csv('data/rolex_df.csv')

## EDA

In [137]:
display(rolex_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 62495 entries, 0 to 66279
Data columns (total 50 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   listing_code                          62495 non-null  object 
 1   brand                                 62495 non-null  object 
 2   model                                 62495 non-null  object 
 3   reference_number                      61846 non-null  object 
 4   movement                              61443 non-null  object 
 5   case_material                         60862 non-null  object 
 6   bracelet_material                     56783 non-null  object 
 7   year_of_production                    62495 non-null  object 
 8   year_is_approximated                  62495 non-null  int32  
 9   condition                             61537 non-null  object 
 10  scope_of_delivery                     62495 non-null  object 
 11  gender              

None

In [138]:
display(rolex_df.describe())

,year_is_approximated,price,number_of_jewels,case_diameter,rating,reviews
count,62495.000000,6.249500e+04,37490.000000,62495.000000,56618.000000,62495.000000
mean,0.121082,3.165010e+04,31.901707,37.127162,4.741688,613.956285
std,0.326225,4.314727e+04,5.082348,4.744927,0.291935,1363.128526
min,0.000000,8.800000e+01,2.000000,6.000000,1.000000,0.000000
25%,0.000000,1.312700e+04,31.000000,36.000000,4.700000,44.000000
50%,0.000000,2.072400e+04,31.000000,40.000000,4.800000,208.000000
75%,0.000000,3.371300e+04,31.000000,40.000000,4.900000,585.000000
max,1.000000,1.506426e+06,200.000000,90.000000,5.000000,8637.000000


We will use only the following columns since they have fewer missing values and have more variation even for the same model. Features that are unrelated to the watch model is especially interesting, such as `condition` and `scope_of_delivery`, as they provide insights on how these factor in to the listing price.

In [139]:
df = rolex_df[['model', 'movement', 'case_material', 'bracelet_material',
               'year_of_production', 'condition', 'scope_of_delivery',
               'country', 'availability', 'case_diameter', 'bezel_material',
               'crystal', 'dial', 'bracelet_color', 'clasp', 'clasp_material',
               'rating', 'reviews']]
df.head(1)

,model,movement,case_material,bracelet_material,year_of_production,condition,scope_of_delivery,country,availability,case_diameter,bezel_material,crystal,dial,bracelet_color,clasp,clasp_material,rating,reviews
0,Datejust 41,Automatic,Gold/Steel,Gold/Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",United States of America,Item is in stock,41,Rose gold,Sapphire crystal,Silver,Gold/Steel,Fold clasp,Gold/Steel,4.2,11


In [141]:
df.shape

(62495, 18)

In [153]:
# alt.Chart(df).mark_bar().encode(
#      alt.X(alt.repeat()).type('quantitative').bin(maxbins=40),
#      y='count()',
# ).properties(
#     width=300,
#     height=200
# ).repeat(
#     ['model', 'movement']
# )

alt.Chart(df).mark_bar().encode(
    y=alt.Y('model:N', sort='-x'),
    x=alt.X('count():Q', title='Count')
)

alt.Chart(...)

In [156]:
plot_columns = df.columns.to_list()

for column in plot_columns:
    top_categories = df[column].value_counts().head(10).index
    filtered_df = df[df[column].isin(top_categories)]

    chart = alt.Chart(filtered_df).mark_bar().encode(
        y=alt.Y(f"{column}:N", sort='-x'),
        x=alt.X('count()', title='Count')
    ).properties(
        title=f"Top 10 Categories in {column}"
    )
    
    chart.display()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)